<a href="https://colab.research.google.com/github/erin-carroll14/AspenRS/blob/main/ploidy_RF_ensemble_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import earth engine library
import ee

# establish connection to own earth engine account
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=plcQ-ic0HxgBCvrDuXqeO9pn2DjnQl1JD9bYsd498KY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6Z8IDFZejjuhkYA998DjMxnz9rkazbrxu1h0Ugf1nJ8ODOXbJg_00

Successfully saved authorization token.


In [ ]:
# call 2019 imagery stack and aspen mask
%cd /content/drive/My\ Drive/Colab\ Notebooks/Aspen/Cytotype

from S2SRstack2019 import stacked2019_JunOct, Colorado
from dist_rf_rmbl_2019 import aspenMask2019, stackVegRMBL2019
from RFensemble_ploidy import deploy, RFmodelPloidy

/content/drive/My Drive/Colab Notebooks/Aspen/Cytotype
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=4AcEifoukqee6LAbZETcGwnnUmd2IjkzSX44etQ0GYY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4UgJbwJ_6-1ztG2XuaKzYKQ-y1TMbn5mlvvZlah41yPusaskOCAhY

Successfully saved authorization token.
stacked2019 loaded
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/con

In [ ]:
# clip imagery to aspen mask
stackVegRMBLaspen2019 = stacked2019_JunOct.mask(aspenMask2019.select('classification'))

In [ ]:
# prepare reference data

# import aspen points with ploidy field attached
aspen = ee.FeatureCollection('users/erin_carroll/aspenAssets/refAspenPloidy')
dip = aspen.filter(ee.Filter.eq('ploidy', 2))
trip = aspen.filter(ee.Filter.eq('ploidy', 3))

# subset validation and training samples (25%, 75%)
random2 = dip.randomColumn('x')
validation2 = random2.filter(ee.Filter.lt('x', 0.25))
training2 = random2.filter(ee.Filter.gte('x', 0.25))

random3 = trip.randomColumn('x')
validation3 = random3.filter(ee.Filter.lt('x', 0.25))
training3 = random3.filter(ee.Filter.gte('x', 0.25))

validation = validation2.merge(validation3)
training = training2.merge(training3)

In [ ]:
# train on 2019 veg imagery
result = RFmodelPloidy(training2, training3, stackVegRMBL2019, stackVegRMBLaspen2019)
print('success')

success


In [ ]:
# accuracy assessment
trainResult = result.sampleRegions(collection=training, properties=['ploidy'], scale=10)
valResult = result.sampleRegions(collection=validation, properties=['ploidy'], scale=10)

In [ ]:
# set up export (printing directly to console not working - error 'too many concurrent aggregations'. Export and then imported as suggested)
expT = ee.batch.Export.table.toDrive(collection=trainResult,
                                     description='trainResult_2019',
                                     fileFormat='CSV')
expV = ee.batch.Export.table.toDrive(collection=valResult,
                                     description='valResult_2019',
                                     fileFormat='CSV')

expT.start()
expV.start()

In [ ]:
# when 'state': 'COMPLETED', move on to next cell to re-import
print(expT.status())
print(expV.status())

{'state': 'COMPLETED', 'description': 'trainResult_2019', 'creation_timestamp_ms': 1610575980058, 'update_timestamp_ms': 1610576174730, 'start_timestamp_ms': 1610576038854, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/'], 'attempt': 1, 'id': '2SEBQDSHYR47WMVWEITSHY2J', 'name': 'projects/earthengine-legacy/operations/2SEBQDSHYR47WMVWEITSHY2J'}
{'state': 'COMPLETED', 'description': 'valResult_2019', 'creation_timestamp_ms': 1610575980443, 'update_timestamp_ms': 1610576252871, 'start_timestamp_ms': 1610576092562, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/'], 'attempt': 1, 'id': 'JZ7OYYZSGC5WIAX4EXSBG4QA', 'name': 'projects/earthengine-legacy/operations/JZ7OYYZSGC5WIAX4EXSBG4QA'}


In [ ]:
# import results from drive

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# important: update links each time!
linkT = 'https://drive.google.com/file/d/15IXUsIMPtTXz5G7B4HtmOLJDhd0R4TvN/view?usp=sharing'
linkV = 'https://drive.google.com/file/d/1U7IJPh1zqdNjbL_e9GyBAjL61SxTe6rt/view?usp=sharing'

import pandas as pd 
  
# to get the id part of the file 
idT = linkT.split("/")[-2] 
idV = linkV.split("/")[-2] 

downloadedT = drive.CreateFile({'id':idT})  
downloadedT.GetContentFile('trainResult_2019.csv')   
trainDF = pd.read_csv('trainResult_2019.csv') 

downloadedV = drive.CreateFile({'id':idV})  
downloadedV.GetContentFile('valResult_2019.csv')   
valDF = pd.read_csv('valResult_2019.csv') 

trainDF['result'] = trainDF['result'].astype(int)
valDF['result'] = trainDF['result'].astype(int)


In [ ]:
from sklearn import metrics
print('TRAINING DATASET')
print(metrics.confusion_matrix(trainDF['ploidy'], trainDF['result'], labels=[2,3]))
print(metrics.classification_report(trainDF['ploidy'], trainDF['result'], labels=[2,3]), '\n')
print('VALIDATION DATASET')
print(metrics.confusion_matrix(valDF['ploidy'], valDF['result'], labels=[2,3]))
print(metrics.classification_report(valDF['ploidy'], valDF['result'], labels=[2,3]))

TRAINING DATASET
[[ 49   0]
 [ 68 240]]
              precision    recall  f1-score   support

           2       0.42      1.00      0.59        49
           3       1.00      0.78      0.88       308

    accuracy                           0.81       357
   macro avg       0.71      0.89      0.73       357
weighted avg       0.92      0.81      0.84       357
 

VALIDATION DATASET
[[13  0]
 [37 52]]
              precision    recall  f1-score   support

           2       0.26      1.00      0.41        13
           3       1.00      0.58      0.74        89

    accuracy                           0.64       102
   macro avg       0.63      0.79      0.58       102
weighted avg       0.91      0.64      0.70       102



In [ ]:
import folium
from folium import plugins

# prepare mapping capabilities (add basemaps and define mapping function)

# Add basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    )
}

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# prepare map

# create a folium map object, centered at RMBL
m = folium.Map(location=[38.9587,-106.9878], zoom_start=11)

# set visualization parameters
rgbVis = {'min': 0.0, 'max': 5000, 'bands': ['B4', 'B3', 'B2']}
vis = {'min':2, 'max':3, 'palette':'red, green'}

In [ ]:
basemaps['Google Satellite'].add_to(m)

m.add_ee_layer(result, {'bands':'confidence', 'min':0, 'max':1, 'palette':'red, yellow, green'}, 'Confidence')
m.add_ee_layer(result, {'bands':'result', 'min':2, 'max':3, 'palette':'#41A7FA, #FA605A'}, 'Ploidy Classification')
m.add_ee_layer(dip, {'color': 'blue'}, 'Diploid')
m.add_ee_layer(trip, {'color': 'red'}, 'Triploid')

# add a layer control panel to the map
m.add_child(folium.LayerControl())

# add fullscreen button
plugins.Fullscreen().add_to(m)

# display the map
display(m)
